
# vLLM Tutorial: Quick Start on Google Colab
This guide walks you through setting up and running offline batched inference with vLLM on Google Colab.

**Running Offline Batched Inference on Colab**

Step 1: Install Required Packages

Start by installing vLLM and other necessary packages using pip:

In [ ]:
!pip install vllm torch  triton

**Step 2: Set Up GPU Environment**

Google Colab offers free access to a Tesla T4 GPU (15GB RAM), which is sufficient for running vLLM. In this tutorial, we'll use the GPT-2 model and set the `gpu_memory_utilization` parameter to 0.5. This ensures that the key-value cache reserved for model inference does not exceed 50% of the total GPU memory. Without this setting, you might encounter out-of-memory errors due to the large key-value cache required for the maximum number of tokens supported by GPT-2.
With `gpu_memory_utilization`=0.5, the model weights will use about 0.24GB, and the key-value cache will require approximately 6.52GB of GPU memory.



**Step 3: Load the Model and Generate Outputs**

Below is the Python script to load the GPT-2 model, define prompts, and generate text outputs:

In [ ]:
from vllm import LLM, SamplingParams

# Load a pre-trained Hugging Face model
model_name = "gpt2"  # Replace with your desired model

# Create an LLM.
#gpu_memory_utilization: The ratio (between 0 and 1) of GPU memory to
#reserve for the model weights, activations, and KV cache. Higher
#values will increase the KV cache size and thus improve the model's
#throughput. However, if the value is too high, it may cause out-of-
# memory (OOM) errors.
llm = LLM(model_name, gpu_memory_utilization = 0.5)

# Sample prompts.
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.generate(prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")


**Step 4: Monitor GPU Memory Usage**

Use the nvidia-smi command to check GPU memory usage. As shown, approximately 7.5GB out of 15GB of GPU memory is used.

In [ ]:
!nvidia-smi

Sun Dec  1 04:51:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0              26W /  70W |   7525MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**Sample Output**

The output of the script may look like this:

```
Prompt: 'Hello, my name is', Generated text: ' Scott. If you want to talk to me, you can. I am a'
Prompt: 'The president of the United States is', Generated text: ' a trained lawyer with experience in working with the vast majority of lawyers who are part'
Prompt: 'The capital of France is', Generated text: ' on the basis of the basis of the shared law. This is called the Common'
Prompt: 'The future of AI is', Generated text: ' becoming a bit more difficult. The big, big picture lies with AI. We'
```

**Step 5: Handle GPU Memory for Repeated Runs**

If you run the script again without restarting the Colab instance, you might encounter out-of-memory errors. To free up GPU memory, you can simply restart the Colab environment using the following command:

In [ ]:
import os
os._exit(00)

For more details, refer to the vLLM Documentation.